In [1]:
import numpy as np
import cv2
import time
import os
import yaml

import open3d as o3d
from open3d.visualization.rendering import OffscreenRenderer, MaterialRecord

import wire_detection_utils as wdu
import viz_utils as vu

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open('wire_detect_config.yaml', 'r') as file:
    detection_config = yaml.safe_load(file)

# Create a WireDetector instance
wire_detector = wdu.WireDetector(detection_config)
input_image_size = [480, 270]

# folder = "/media/tyler/Storage/field_tests/street_wire_experiments/wire_tracking_05-07_40fov/" 
folder = '/media/tyler/Storage/Research/Datasets/wire_tracking_05-07_40fov/'
# folder = "/home/tyler/Documents/wire_tracking_05-07_40fov/" 
# folder = "/media/tyler/Storage/field_tests/hawkins_2-19/wire_tracking_02-19_1/" 
# folder = "/media/tyler/Storage/field_tests/acfa_3-21/" 

rgb_folder = folder + 'rgb_images/'
depth_folder = folder + 'depth_images/'
camera_intrinsics_file = folder + 'rgb_camera_intrinsics.npy'
camera_intrinsics = np.load(camera_intrinsics_file)
rgb_file_list = sorted(os.listdir(rgb_folder))
depth_file_list = sorted(os.listdir(depth_folder))
original_image_size = cv2.imread(os.path.join(rgb_folder, rgb_file_list[0])).shape[:2][::-1]  # (width, height)

# resize the camera intrinsics to match the input image size
camera_intrinsics[0, 0] *= input_image_size[0] / original_image_size[0]
camera_intrinsics[1, 1] *= input_image_size[1] / original_image_size[1]
camera_intrinsics[0, 2] *= input_image_size[0] / original_image_size[0]
camera_intrinsics[1, 2] *= input_image_size[1] / original_image_size[1]

wire_detection_dir = folder + 'ransac_results/'

if os.path.exists(wire_detection_dir) == False:
    os.makedirs(wire_detection_dir)

In [4]:

avg_frequency = 0.0
renderer, material = vu.create_renderer()

for file in rgb_file_list:
    start_time = time.perf_counter()
    rgb_timestamp = int(file.split('.')[0])
    closest_depth_timestamp = None
    for depth_file in depth_file_list:
        if closest_depth_timestamp is None or abs(int(depth_file.split('.')[0]) - rgb_timestamp) < abs(int(closest_depth_timestamp.split('.')[0]) - rgb_timestamp):
            closest_depth_timestamp = depth_file
            closest_depth_file = os.path.join(depth_folder, str(closest_depth_timestamp)) + '.npy'
            
    rgb_img = cv2.imread(os.path.join(rgb_folder, file))
    depth_img = np.load(os.path.join(depth_folder, str(closest_depth_timestamp)))

    rgb_img = cv2.resize(rgb_img, (input_image_size[0], input_image_size[1]))
    depth_img = cv2.resize(depth_img, (input_image_size[0], input_image_size[1]))
    
    min_depth = 0.5
    depth_img[depth_img <= min_depth] = 0

    start_time = time.perf_counter()
    wire_lines, wire_midpoints, avg_angle, midpoint_dists_wrt_center = wire_detector.detect_wires_2d(rgb_img)

    regions_of_interest, roi_line_counts = wire_detector.find_regions_of_interest(depth_img, avg_angle, midpoint_dists_wrt_center)

    fitted_lines, roi_pcs, roi_point_colors, masked_rgb_img = wire_detector.ransac_on_rois(regions_of_interest, roi_line_counts, avg_angle, depth_img, camera_intrinsics, viz_img=rgb_img)

    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    avg_frequency += 1.0 / elapsed_time

    for fitted_line in fitted_lines:
        masked_rgb_img = vu.draw_3d_line_on_image(masked_rgb_img, fitted_line, camera_intrinsics, color=(0, 255, 0), thickness=2)

    img_name_3d = os.path.join(wire_detection_dir, str(rgb_timestamp) + '_3d.png')
    vu.capture_fitted_lines_in_image(renderer, material, img_name_3d, fitted_lines, roi_pcs, roi_point_colors)
    img_name_2d = os.path.join(wire_detection_dir, str(rgb_timestamp) + '_2d.png')
    masked_rgb_img = cv2.resize(masked_rgb_img, (1920, 1080))  # Resize to 1280x720 for better visualization
    cv2.imwrite(img_name_2d, masked_rgb_img)

avg_frequency /= len(rgb_file_list)
print("Average frequency: ", avg_frequency)


Processed 1746650644465219840.png in 0.13 seconds. Average angle: 1.59 degrees.
Processed 1746650644531419968.png in 0.07 seconds. Average angle: 1.55 degrees.
Processed 1746650644591884672.png in 0.11 seconds. Average angle: 1.59 degrees.
Processed 1746650644626915968.png in 0.14 seconds. Average angle: 1.60 degrees.
Processed 1746650644658027136.png in 0.16 seconds. Average angle: 1.59 degrees.
Processed 1746650644703578496.png in 0.13 seconds. Average angle: 1.61 degrees.
Processed 1746650644755096640.png in 0.14 seconds. Average angle: 1.60 degrees.
Processed 1746650644798742464.png in 0.11 seconds. Average angle: 1.60 degrees.
Processed 1746650644835528416.png in 0.12 seconds. Average angle: 1.59 degrees.
Processed 1746650644872605152.png in 0.10 seconds. Average angle: 1.59 degrees.
Processed 1746650644920445728.png in 0.14 seconds. Average angle: 1.59 degrees.
Processed 1746650644965673024.png in 0.17 seconds. Average angle: 1.59 degrees.
Processed 1746650645013071168.png in 0.1

TypeError: 'NoneType' object is not iterable